# boiler plate shit

In [1]:
from maze_env import MazeEnv_v0
from utils.PettingZooEnv_new import PettingZooEnv_new
from utils.training_utils import *
import supersuit
import numpy as np
import tianshou as ts
import torch
import torch.nn as nn
from collections import deque, Counter

from abstraction import generate_abstractions
from policy import DQNPolicy_new
from utils.training_utils import *

In [2]:
eps_train, eps_test = 0.95, 0.0 # exploration rate for training and testing respectively
eps_decay, eps_min = 0.999, 0.15 # the exploration rate decay and the minimum exploration rate
lr, epochs, batch_size = 5e-4, 150, 512 # the learning rate, max epochs per new maze intro and the update batch size
gamma, n_step, target_update_freq = 0.9, 3, 100 # gamma in dqn formula, number of steps to look ahead, number of update calls before updating target network
train_num, test_num = 10, 1 # num of simultaneous training and testing environments respectively
buffer_size = 30000 # buffer size
step_per_epoch, step_per_collect, ep_per_collect = 10000, 200, 1 # number of steps for each epoch, number of steps to collect before updating, number of episodes before updating
maze_width = 6 # maze width (not incl. walls)
high_eps_run, obs_train, passed_mazes = False, True, True # for random high eps run, for interleaving (might be broken?), whether the policies passed the mazes
steps_total, steps_n = 0, 0 # steps count total, steps count within epoch
n_mazes, total_mazes = 0, 26 # start with 3 (it will add one later) mazes initially to prevent single maze overfitting, total number of random mazes
# for the trivial maze, we use 36 (since it should be 'easier')
test_mazes = [] # for printing later
threshold_rew = 0.5 # threshold reward to consider a maze passed (tentative value)

max_actions = 8 # number of abstractions is: max_actions-5

In [3]:
# get the vectorized training/testing environments
train_envs = ts.env.DummyVectorEnv([preprocess_maze_env for _ in range(train_num)])
test_envs = ts.env.DummyVectorEnv([preprocess_maze_env for _ in range(test_num)])

# set up training with no render environment
env = preprocess_maze_env()

# set up human render environment
env_human = preprocess_maze_env(render_mode="human")
env_human = ts.env.DummyVectorEnv([lambda: env_human])

# get agent names
agents = env.agents

# observation spaces/action spaces for the two agents
state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n

# define DQN network (128x3 hidden units linear)
#net_obs = Net(state_shape, action_shape, [128,128,128])
#net_obs = Net(state_shape, action_shape, [512, 512, 512])
net_obs = CNN(maze_width=6, max_actions=max_actions)
optim_obs = torch.optim.Adam(params=net_obs.parameters(), lr=lr)

#net_exp = Net(state_shape, action_shape, [8])
#optim_exp = torch.optim.Adam(params=net_exp.parameters(), lr=lr)

# set up policy and collectors
agent_observer = ts.policy.DQNPolicy(net_obs, optim_obs, gamma, n_step, target_update_freq)
#agent_explorer = ts.policy.DQNPolicy(net_exp, optim_exp, gamma, n_step, target_update_freq)
#agent_policies = [agent_observer, agent_explorer]
#agent_policies = [ts.policy.RandomPolicy(), ts.policy.RandomPolicy()] # baseline testing
#policy = ts.policy.MultiAgentPolicyManager(agent_policies, env)

policy = agent_observer

# define the training collector (the calc q and step functions)
train_collector = ts.data.Collector(
    policy, 
    train_envs, 
    ts.data.VectorReplayBuffer(buffer_size, train_num),
    exploration_noise=True
)

abstraction_buffer = ts.data.ReplayBuffer(100000)
episode_history = deque(maxlen=500) # 500 most recent successful episodes

f:\Desktop\University\Exeter\ECMM451 Final Year Project\Final_Year_Project\maze_env\envs\mazelib.py:32: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seed)
f:\Desktop\University\Exeter\ECMM451 Final Year Project\Final_Year_Project\maze_env\envs\mazelib.py:32: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seed)


In [58]:
policy.load_state_dict(torch.load("model/trivial_maze_baseline_20-6-23.pt"))

<All keys matched successfully>

---

# policy testing for mapped actions

In [2]:
mydict = {key:0 for key in range(5, 7)}

In [50]:
mydict[5] = "pp"

In [51]:
mydict

{5: 'pp', 6: 0}

In [48]:
key=7
if mydict.get(key) == 0:
    print("no abs")
elif mydict.get(key) == None:
    print("is none")
else:
    print("asdasd")

is none


In [61]:
myarr = np.array([6])
assert mydict[myarr[0]], "bad key"
print(mydict[myarr[0]])

AssertionError: bad key

In [9]:
abstraction = np.array([1,1,1,1,1])
mydict[5] = abstraction

In [14]:
search_abs = np.array([1,1,1,1,1])
for action, abstraction in mydict.items():
    if np.all(abstraction == search_abs):
        print(action)

5


---
# environment testing

In [4]:
actions = np.array([[0,1], [1,0], [1,0]])
direction = np.array([0,0])

for action in actions:
    direction += action
print(direction)

[2 1]


In [12]:
obs, done = env_human.reset()
batch = ts.data.Batch(obs = obs)
batch.info = {}
env_human.render()

[None]

In [6]:
act = np.array([1])
obs, rew, truncated, terminated, info  = env_human.step(act)
batch = ts.data.Batch(obs = obs)
batch.info = info
print("Action:", act)
print(batch)

Action: [1]
Batch(
    obs: Batch(
             agent_id: array(['observer'], dtype=object),
             mask: array([[False,  True,  True, False, False]]),
             obs: array([[[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
                           [1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
                           [1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.],
                           [1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1.],
                           [1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1.],
                           [1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.],
                           [1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1.],
                           [1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
                           [1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1.],
                           [1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.],
                           [1., 

In [11]:
(1, 0) + np.array([1, 1])

array([2, 1])

In [14]:
abstraction = [np.array([1,1,4,4,1,1,3,3,1,1])]
obs, rew, truncated, terminated, info  = env_human.step(abstraction)
batch = ts.data.Batch(obs = obs)
batch.info = info
print("Action:", act)
print(batch)

Action: [1]
Batch(
    obs: Batch(
             agent_id: array(['observer'], dtype=object),
             mask: array([[False, False,  True, False,  True]]),
             obs: array([[[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
                           [1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
                           [1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.],
                           [1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1.],
                           [1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1.],
                           [1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.],
                           [1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1.],
                           [1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
                           [1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1.],
                           [1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.],
                           [1., 

In [8]:
def check_legal_moves_abstractions(batch: ts.data.Batch, action_keys, abstraction_mask):
    """
    We cannot check the legal moves in the environment due to some limitations with how the wrappers work,
    So we have to check it in the policy itself.
    This is kinda a half-assed way to do it, but whatever
    """
    current_loc = np.where(batch.obs[1,:,:]==1)
    obs = batch.obs[0,:,:]

    # check the abstractions to see if they're valid
    for key in action_keys:
        wall = False
        abstraction = abstractions[key]
        current_loc1 = np.copy(current_loc).reshape(2)

        # iterate through the actions one-by-one to see if if its valid
        for action in abstraction:
            direction = action_to_direction[action]
            # find new location
            new_loc = current_loc1 + direction
            location = obs[tuple(new_loc)]
            if int(location) == 1: # wall
                abstraction_mask[key-5] = False
                wall = True
                break # if encouter wall, stop the abstraction validity search and set flag to true
            else:
                current_loc1 = new_loc # otherwise keep going

        if wall:
            continue
        else:
            # if no wall flag, the abstraction is valid.
            abstraction_mask[key-5] = True
    
    return abstraction_mask

def abstraction_action_mask(batch: ts.data.Batch):
    # assume the mask is true/false boolean
    abstraction_mask = np.full(7-5, False) # make an array with all false
    keys = 5 # get the used actions keys (i.e. they have abstractions)

    # check the legal moves of the abstractions
    abstraction_mask = check_legal_moves_abstractions(batch, keys, abstraction_mask)

    # modify the original mask
    batch.obs.mask[0] = np.append(batch.obs.mask[0], abstraction_mask)

action_to_direction = {
            0: np.array([0,0]), # no movement
            1: np.array([-1,0]), # up
            2: np.array([1,0]), # down
            3: np.array([0,1]), # right
            4: np.array([0,-1]) # left
        }

In [ ]:
abstractions = {key:0 for key in range(5, 7)}
abstraction[5] = np.array([1,1])

---
# policy testing

In [6]:
policy = DQNPolicy_new(8, net_obs, optim_obs, gamma, n_step, target_update_freq)
collector_policy = ts.data.Collector(policy, env_human, ts.data.VectorReplayBuffer(buffer_size, train_num), exploration_noise=True)

In [51]:
key = policy.add_abstraction(np.array([1,1,1]))

Max actions reached. Cannot append any additional abstractions.


In [16]:
policy.remove_abstraction(5)

In [52]:
print(policy.available_action_keys())
print(policy.used_action_keys())

[]
[5, 6, 7]


In [53]:
for i in range(5, 8):
    print(policy.abstractions[i])

[array([1, 1])]
[array([1, 3])]
[array([1, 4])]


In [26]:
policy.map_action(np.array([5]))

[array([1, 1])]

In [7]:
obs, done = env_human.reset()
batch = ts.data.Batch(obs = obs)
batch.info = {}
env_human.render()
policy.set_eps(1)

[None]

In [19]:
policy.set_eps(1)

In [49]:
collector_policy.collect(n_step=1, render=1/120)

before ----
 Batch(
    obs: Batch(
             agent_id: array(['observer'], dtype=object),
             obs: array([[[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
                           [1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
                           [1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.],
                           [1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1.],
                           [1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1.],
                           [1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.],
                           [1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1.],
                           [1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
                           [1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1.],
                           [1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.],
                           [1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1.],
               

{'n/ep': 0,
 'n/st': 1,
 'rews': array([], dtype=float64),
 'lens': array([], dtype=int32),
 'idxs': array([], dtype=int32),
 'rew': 0,
 'len': 0,
 'rew_std': 0,
 'len_std': 0}

In [45]:
policy.abstractions[7]

[array([1, 4])]

In [12]:
mask = np.array([[False,  True,  True, False, False]])
abstraction_mask = np.array([False, False, False])

mask= np.array([np.concatenate([mask[0], abstraction_mask])])
mask

array([[False,  True,  True, False, False, False, False, False]])

---
# sleeping & Dreaming

In [59]:
mazes = 30
policy.set_eps(0.4)
train_collector.reset()
episode_history = deque(maxlen=500) # 500 most recent successful episodes

for _ in range(10):
    train_collector.reset_env(gym_reset_kwargs={"options":{"maze_type":"trivial", "n_mazes":mazes, "random":True}})
    result = train_collector.collect(n_episode=ep_per_collect, gym_reset_kwargs={"options":{"maze_type":"trivial", "n_mazes":mazes, "random":True}})

    # add the successful episode steps into the episode history
    if result['rews'] > threshold_rew:
        episode_history.append(train_collector.buffer[int(result['idxs']):int(result['idxs'] + result['lens'])])

abstraction = generate_abstractions(episode_history)

Best abstraction found: 433


In [111]:
current_loc = np.where(train_collector.buffer[0].obs.obs[1,:,:]==1)
current_loc1 = np.copy(current_loc).reshape(2)
obs = train_collector.buffer[0].obs.obs[0,:,:]

In [107]:
direction = np.array([0,1])

In [108]:
print(current_loc1)
print(direction)

[0 1]
[0 1]


In [110]:
new_loc = current_loc1 + direction
print(new_loc)

[0 2]


In [73]:
abstraction
abstraction1 = np.copy(abstraction)

In [46]:
def is_sub_arr_np(a1, a2):
    """
    Credits to: https://stackoverflow.com/questions/57004175/numpy-check-if-1-d-array-is-sub-array-of-another
    """
    l1, = a1.shape
    s1, = a1.strides
    l2, = a2.shape
    a1_win = np.lib.stride_tricks.as_strided(a1, (l1 - l2 + 1, l2), (s1, s1))
    any_sub = np.any(np.all(a1_win == a2, axis=1))

    if any_sub:
        indices = np.where(np.all(a1_win == a2, axis=1)==True)[0][0] # return the first indices where it matches
        return any_sub, [indices, indices+l2]
    else:
        return any_sub, None

In [63]:
def append_new_action(action_history, abstraction_number, indices):
    """
    action history: the successful episode history's action array
    abstraction_number: the number representation of the new abstraction for the agent
    """
    return np.concatenate([action_history[:indices[0]], np.array([abstraction_number]), action_history[indices[-1]:]])

In [96]:
new_actions = []
for i in range(len(episode_history)):
    # get the episode actions
    ep_act = episode_history[i].act

    is_sub = True
    any_change = False
    while is_sub:
        is_sub, indices = is_sub_arr_np(ep_act, abstraction)
        if is_sub:
            # the 5 NEEDS TO BE CHANGED TO MODEL NUMBER REPRESENTATION
            ep_act = append_new_action(ep_act, 5, indices)
            any_change = True

    if any_change:
        new_actions.append(ep_act)
        any_change = False

print(new_actions)

[array([1, 1, 1, 4, 2, 2, 1, 1, 3, 2, 2, 4, 1, 2, 1, 4, 1, 1, 2, 5, 2, 1,
       1, 3, 2, 1, 2, 1, 3, 3, 3, 4, 3, 3, 3, 1, 3, 3, 1, 1], dtype=int64), array([1, 1, 4, 5, 1, 1, 3, 1, 3, 1, 4, 1], dtype=int64), array([1, 1, 2, 5, 3, 1, 1, 3, 4, 3, 2, 4, 1, 1, 3, 2, 1, 3, 3, 1, 2, 1,
       1, 1], dtype=int64), array([1, 4, 4, 5, 1, 4, 4, 1, 1, 4, 1, 3, 3, 4, 4, 2, 1, 4, 4, 4, 1, 1],
      dtype=int64), array([1, 1, 1, 1, 4, 2, 3, 5, 1, 1, 3, 3, 1, 3, 4, 1], dtype=int64), array([1, 3, 5, 1, 1, 3, 1, 1, 2, 2, 1, 1, 1, 4, 3, 1], dtype=int64), array([1, 4, 3, 4, 4, 1, 4, 5, 4, 4, 4, 1, 4, 1, 1, 1, 4, 1], dtype=int64), array([5, 1, 3, 3, 1, 3, 1, 1, 4, 1, 4, 1, 3, 1], dtype=int64), array([3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 4, 3, 3,
       3, 3, 1, 2, 5, 3, 1, 3], dtype=int64), array([1, 1, 1, 1, 3, 2, 5, 1, 1, 1, 4, 3, 4, 1], dtype=int64), array([5, 4, 1, 1, 1, 4, 1, 1, 1, 3, 4, 1], dtype=int64), array([1, 4, 4, 4, 5, 1, 4, 4, 4, 1, 3, 4, 4, 1, 1, 1, 4, 1], dtype=int64), 

In [113]:
maze = preprocess_maze_env(render_mode='human', size=6)
maze.reset()
maze.render()

f:\Desktop\University\Exeter\ECMM451 Final Year Project\Final_Year_Project\maze_env\envs\mazelib.py:32: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seed)


In [114]:
action = 1
obs = maze.step(action)

({'agent_id': 'observer',
  'obs': array([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
          [1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.],
          [1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1.],
          [1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1.],
          [1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.],
          [1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1.],
          [1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
          [1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1.],
          [1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],
  
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
   

In [110]:
agent_id, act,  = 

({'agent_id': 'observer',
  'obs': array([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
          [1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.],
          [1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1.],
          [1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1.],
          [1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.],
          [1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1.],
          [1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
          [1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.],
          [1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1.],
          [1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],
  
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
   

In [111]:
episode_history[0]


Batch(
    obs: Batch(
             obs: array([[[[1., 1., 1., ..., 1., 1., 1.],
                           [1., 0., 0., ..., 0., 0., 1.],
                           [1., 0., 0., ..., 0., 0., 1.],
                           ...,
                           [1., 0., 0., ..., 0., 0., 1.],
                           [1., 0., 0., ..., 0., 0., 1.],
                           [1., 1., 1., ..., 1., 1., 1.]],
                  
                          [[0., 0., 0., ..., 0., 0., 0.],
                           [0., 0., 0., ..., 0., 0., 0.],
                           [0., 0., 0., ..., 0., 0., 0.],
                           ...,
                           [0., 0., 0., ..., 0., 0., 0.],
                           [0., 0., 0., ..., 0., 0., 0.],
                           [0., 0., 0., ..., 0., 0., 0.]],
                  
                          [[0., 0., 0., ..., 0., 1., 0.],
                           [0., 0., 0., ..., 0., 0., 0.],
                           [0., 0., 0., ..., 0., 0., 0.],
   

In [ ]:
new_ep_hist2 = np.concatenate([ep_act[:indices[0]], np.array([5]), ep_act[indices[-1]:]])
print(new_ep_hist)

In [20]:
a1 = ep_act
a2 = abstraction

l1, = a1.shape
s1, = a1.strides
l2, = a2.shape
a1_win = np.lib.stride_tricks.as_strided(a1, (l1 - l2 + 1, l2), (s1, s1))
#np.any(np.all(a1_win == a2, axis=1))

In [27]:
a1_win

array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 2],
       [1, 1, 2, 1],
       [1, 2, 1, 1],
       [2, 1, 1, 2],
       [1, 1, 2, 2],
       [1, 2, 2, 4],
       [2, 2, 4, 1],
       [2, 4, 1, 3],
       [4, 1, 3, 1],
       [1, 3, 1, 1],
       [3, 1, 1, 3],
       [1, 1, 3, 1],
       [1, 3, 1, 1],
       [3, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 2],
       [1, 1, 2, 1],
       [1, 2, 1, 4],
       [2, 1, 4, 1]], dtype=int64)

In [30]:
indices = np.where(np.all(a1_win == a2, axis=1)==True)[0]
indices

array([ 0,  1, 16], dtype=int64)

In [15]:
is_sub_arr_np(episode_history[1].act, abstraction)

True

In [18]:
ep_act

In [83]:
np.array(list(LCS_counter.keys()))[np.argsort(-np.array(ratings))]

array(['121', '44444', '444444', '1111', '44244', '44414', '12', '11111',
       '2444', '111', '1211', '2111', '21111', '1114', '444', '11141',
       '1444', '434', '112111', '212', '123', '11114', '121111', '42',
       '422', '111111', '333333', '4441', '444414', '41114', '412', '21',
       '4444144', '444441', '442', '244', '11444', '31111', '11113',
       '321', '211', '421', '1444444', '411141', '24', '144', '144444',
       '2322', '4444444', '133', '1311', '4144', '44441', '111141', '14',
       '4442', '224', '312', '12111', '344', '31', '1112', '111114',
       '4444', '111444', '222242', '113', '22242', '4443', '4411', '4111',
       '2121', '124', '1244', '33333', '2224', '43444', '342', '43333',
       '112', '1121111', '3444', '4444441', '14414', '11211', '41444',
       '441444', '214', '142', '11', '334', '441', '233', '1141', '221',
       '232', '44434', '34', '11414', '2222224', '1121', '243', '414444',
       '144414', '2223', '41444444', '433', '343', '32', '333

In [12]:
[[0 for k in range(len('asdad'))] for l in range(len('asdasdada'))]

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]